In [ ]:
import pandas as pd
import numpy as np

# 데이터 불러오기 1. 횡단보도
ped = pd.read_csv('./data/횡단보도_완료.csv')

# 데이터 불러오기 2. 새로운 데이터프레임
another = pd.read_csv('./data/crossroad.csv', encoding = 'cp949')

In [ ]:
### 필요할 때만 할 것
# 좌표 변환 처리, 변수명 확인해서 데이터프레임마다 처리 할 것
another = another.loc[another['구코드 (공통);'] == 500][['교차로코드', 'X좌표', 'Y좌표']]

from pyproj import Proj
from pyproj import transform

WGS84 = {'proj':'latlong', 'datum':'WGS84', 'ellps':'WGS84'}

GRS80 = {'proj':'tmerc', 'lat_0':'38', 'lon_0':'127', 'k':1, 'x_0':200000,
         'y_0':600000, 'ellps':'GRS80', 'units':'m'}

# 좌표변환 함수
def grs80_to_wgs84(x, y):
   return transform( Proj(**GRS80), Proj(**WGS84), x, y)


# 위도, 경도 설정 잘 할 것!!
lon, lat = grs80_to_wgs84(cr_df.X좌표, cr_df.Y좌표)

another['lat'] = lat
another['lon'] = lon

another_df = another[['lat', 'lon']]

In [ ]:
# 데이터프레임 병합 함수
def crosswalk_and_newdf(ped, another):
    
    # 합치고자 하는 데이터프레임을 '위도', '경도' 변수만 남겨서 저장할 것
    another.columns = ['new_lat', 'new_lon']

    # 횡단보도
    ped_xy = ped[['crosswalk_id', 'lon', 'lat']]
    ped_xy.columns = ['crosswalk_id', 'ped_lon', 'ped_lat']

    another['key'] = 0
    ped_xy['key'] = 0

    # cross join
    cross = pd.merge(another, ped_xy, on = 'key', how = 'outer')

    # 계산 method
    lat1 = np.deg2rad(cross['new_lat'])
    lat2 = np.deg2rad(cross['ped_lat'])
    lon1 = np.deg2rad(cross['new_lon'])
    lon2 = np.deg2rad(cross['ped_lon'])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    # 거리 계산 (미터 단위)
    cross['distance'] = c * 6371 * 1000
    cross2 = cross[cross['distance'] <= 50]
    howmany = cross2.groupby('crosswalk_id').count().reset_index().iloc[:, 0:2]

    # 
    new_df = ped.drop_duplicates('crosswalk_id')['crosswalk_id']
    new_df = pd.merge(new_df, howmany, on = 'crosswalk_id', how = 'left')
    new_df.fillna(0, inplace = True)
    new_df.columns = ['crosswalk_id', 'count']

    return new_df